In [80]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import pandas as pd

In [81]:

standard_of_care_df = pd.read_excel('./potlako_data_issues.xlsx', 
                                    sheet_name='Standard of care',
                                    usecols='A,B,D',
                                    engine='openpyxl',).dropna()

intervention_df = pd.read_excel('./potlako_data_issues.xlsx', 
                                sheet_name='Intervention',
                                usecols='A,B,D',
                                engine='openpyxl',).dropna()



df = pd.concat([intervention_df, standard_of_care_df])


df = df[df["COMMENT"].str.contains("navi", case=False)]


pids = df['PID '].values.tolist()


data = dict()

data['subject_identifier'] = list()
data['history_depth'] = list()
data['history_objs'] = list()

for pid in pids:
    data['subject_identifier'].append(pid)
    
    history_objs = NavigationSummaryAndPlan.history.filter(subject_identifier=pid)
    
    data['history_depth'].append(history_objs.count())
    
    
pd.DataFrame(data, columns=['subject_identifier', 'history_depth'])



,subject_identifier,history_depth
0,132-40500009-3,100
1,132-40300011-1,91
2,132-40300012-9,91
3,132-40400057-3,2
4,132-40600079-5,2
5,132-40700001-8,99
6,132-40700006-7,94
7,132-40700009-1,92
8,132-40700010-9,88
9,132-40700024-0,94


In [101]:
data = dict()

data['pid'] = list()
data['history_depth'] = list()
data['diagnostic_plan'] = list()

    
for pid in pids:
    diagnostic_plans = NavigationSummaryAndPlan.history.filter(subject_identifier=pid)
    data['pid'].append(pid)
    data['history_depth'].append(diagnostic_plans.count())
    
    
    d_plans_list = list(dict.fromkeys(diagnostic_plans.order_by('history_date').values_list('diagnostic_plan', flat=True)))
    
    if diagnostic_plans.count() > 2:
        latest_navigation_plan = diagnostic_plans.latest('modified')
    else:
    
        latest_navigation_plan = diagnostic_plans.first()
    
    d_plans_list.append(latest_navigation_plan)
    
    data['diagnostic_plan'].append(d_plans_list)
    
    
df1 = pd.DataFrame(data)

df1
    

,pid,history_depth,diagnostic_plan
0,132-40500009-3,100,"[Sputum for MCS, CXR, CXR reporting, Sputum fo..."
1,132-40300011-1,91,"[Penile biopsy collected, plan F/U results. Pa..."
2,132-40300012-9,91,[Collect more hx from RA about what is being d...
3,132-40400057-3,2,[1. Book Breast clinic for possible biopsy\r\n...
4,132-40600079-5,2,[1. Book breast clinic in PMH for possible bio...
5,132-40700001-8,99,"[Physical exam - inguinal lymphadenopathy, ?Re..."
6,132-40700006-7,94,[Patient should be reviewed by surgeon. Way f...
7,132-40700009-1,92,[Plan is for patient do do breast imaging and ...
8,132-40700010-9,88,[Plan is for patient to be seen at surgical cl...
9,132-40700024-0,94,"[Book a mammogram, to confirm the size of the ..."


In [102]:
df2 = pd.DataFrame(data['diagnostic_plan'])

df2

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,"Sputum for MCS, CXR, CXR reporting","Sputum for MCS, MO review, CXR, CXR reporting","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...",...,132-40500009-3 as of 2022-07-15 07:06:30.38900...,None,None,None,None,None,None,None,None,None
1,"Penile biopsy collected, plan F/U results. Pai...","Penile biopsy collected, plan F/U results. Pai...","Penile biopsy collected, plan F/U results. Pai...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...",...,132-40300011-1 as of 2022-08-01 09:28:26.09945...,None,None,None,None,None,None,None,None,None
2,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,132-40300012-9 as of 2022-09-06 08:36:07.03982...,None,None,None,None,None,None,None
3,1. Book Breast clinic for possible biopsy\r\n2...,1. Book Breast clinic for possible biopsy\r\n2...,132-40400057-3 as of 2022-12-12 08:28:25.56376...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1. Book breast clinic in PMH for possible biop...,1. Book breast clinic in PMH for possible biop...,132-40600079-5 as of 2022-11-29 07:45:38.54859...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"Physical exam - inguinal lymphadenopathy, ?Rec...",Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,...,132-40700001-8 as of 2022-07-15 07:06:30.48100...,None,None,None,None,None,None,None,None,None
6,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,...,None,None,None,None,None,None,None,None,None,None
7,Plan is for patient do do breast imaging and r...,Plan is for patient do do breast imaging and r...,Plan is for patient do do breast imaging and b...,Plan is for patient do breast mammogram and bi...,Plan is for patient do breast mammogram/uss an...,Plan is for patient do breast mammogram/uss an...,Plan is for patient do breast mammogram/uss an...,Plan is for patient do breast mammogram/uss an..

In [103]:
df3 = df1.join(df2)

del df3['diagnostic_plan']

df3.to_excel("history_objects.xlsx",
             sheet_name='All History Objects') 


df3

,pid,history_depth,0,1,2,3,4,5,6,7,...,12,13,14,15,16,17,18,19,20,21
0,132-40500009-3,100,"Sputum for MCS, CXR, CXR reporting","Sputum for MCS, MO review, CXR, CXR reporting","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...","Sputum for MCS, MO review, CXR, CXR reporting\...",...,132-40500009-3 as of 2022-07-15 07:06:30.38900...,None,None,None,None,None,None,None,None,None
1,132-40300011-1,91,"Penile biopsy collected, plan F/U results. Pai...","Penile biopsy collected, plan F/U results. Pai...","Penile biopsy collected, plan F/U results. Pai...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...","Penile biopsy collected in gweta, the plan is ...",...,132-40300011-1 as of 2022-08-01 09:28:26.09945...,None,None,None,None,None,None,None,None,None
2,132-40300012-9,91,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,...,Collect more hx from RA about what is being do...,Collect more hx from RA about what is being do...,132-40300012-9 as of 2022-09-06 08:36:07.03982...,None,None,None,None,None,None,None
3,132-40400057-3,2,1. Book Breast clinic for possible biopsy\r\n2...,1. Book Breast clinic for possible biopsy\r\n2...,132-40400057-3 as of 2022-12-12 08:28:25.56376...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,132-40600079-5,2,1. Book breast clinic in PMH for possible biop...,1. Book breast clinic in PMH for possible biop...,132-40600079-5 as of 2022-11-29 07:45:38.54859...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,132-40700001-8,99,"Physical exam - inguinal lymphadenopathy, ?Rec...",Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,Plan is for patient to be reviewed by MO and g...,...,132-40700001-8 as of 2022-07-15 07:06:30.48100...,None,None,None,None,None,None,None,None,None
6,132-40700006-7,94,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,Patient should be reviewed by surgeon. Way fo...,...,None,None,None,None,None,None,None,None,None,None
7,132-40700009-1,92,Plan is for patient do do breast imaging and r...,Plan is for patient do do breast imaging and r...,Plan is for patient do do breast imaging and b...,Plan is for patient do breast mammogram and bi...,Plan is for patient do breast mammogram/uss an...,Plan is for patient do breast mammogram/uss an...,Plan is for patient do breast mammogram/uss an...,Plan is for patient do breast mammogram/uss an...,...,None,None,None,None,None,None,None,None,None,None
8,132-40700010-9,88,Plan is for patient to be seen at surgical cli...,Plan is for patient to be seen at surgical cli...,Plan is for patient to be seen at surgical cli...,Plan is for patient to be seen at surgical cli...,Plan is for patient to be seen at surgical cli...,Plan is for patient to be seen at surgical 

In [104]:
NavigationSummaryAndPlan.history.first().__dict__
# NavigationSummaryAndPlan.objects.first().evaluationtimeline_set.all()

{'_state': <django.db.models.base.ModelState at 0x7f9959120be0>,
 'created': datetime.datetime(2022, 12, 5, 7, 28, 42, 961536, tzinfo=<UTC>),
 'modified': datetime.datetime(2022, 12, 14, 17, 23, 31, 493548, tzinfo=<UTC>),
 'user_created': 'tntloedibe',
 'user_modified': 'kmanyake',
 'hostname_created': 'potlako-plus',
 'hostname_modified': 'potlako-plus',
 'revision': '0.2.20:master:f4568843a8f3a94bfd49e9e7d0f5b998489dcb63',
 'device_created': '99',
 'device_modified': '99',
 'id': UUID('7da81d2f-ea7f-4f30-b193-6ef0c40c19c9'),
 'subject_identifier': '132-40600084-5',
 'diagnostic_plan': '1. Encourage the participant to get HIV test.\r\n2. Book Dermatology clinic for a possible biopsy of the non-healing ulcer.\r\n3. Book surgical clinic for assessment and management of the wound, in case it is Necrotising faciatis.\r\n4. Follow up results if biopsy is done.\r\n5. If results show malignancy, book oncology for treatment plan.\r\n6. if no malignancy, continue with 6 months follow ups.\r\n1